In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [2]:
import numpy as np
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [5]:
data_dir = 'data'

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [7]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test']}

In [8]:
dataset_sizes  = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 793, 'test': 207}

In [9]:
class_names = image_datasets['train'].classes
class_names

['cavalo',
 'comida',
 'dinossauro',
 'elefante',
 'flor',
 'indio',
 'montanha',
 'monumento',
 'onibus',
 'praia']

In [10]:
model = models.resnet18(pretrained=True)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [13]:
def train(model, dataloader, optimizer, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [14]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [15]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model.to(device);

In [18]:
for epoch in range(1, 30+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['test'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.9

Train Epoch: 1 [0/793 (0%)]	Loss: 0.383796
Train Epoch: 1 [640/793 (80%)]	Loss: 0.307263

Test set: Average loss: 0.0090, Accuracy: 194/207 (94%)

Train Epoch: 2 [0/793 (0%)]	Loss: 0.411605
Train Epoch: 2 [640/793 (80%)]	Loss: 0.582151

Test set: Average loss: 0.0097, Accuracy: 190/207 (92%)

Train Epoch: 3 [0/793 (0%)]	Loss: 0.376951
Train Epoch: 3 [640/793 (80%)]	Loss: 0.245059

Test set: Average loss: 0.0096, Accuracy: 194/207 (94%)

Train Epoch: 4 [0/793 (0%)]	Loss: 0.512046
Train Epoch: 4 [640/793 (80%)]	Loss: 0.200015

Test set: Average loss: 0.0100, Accuracy: 191/207 (92%)

Train Epoch: 5 [0/793 (0%)]	Loss: 0.361526
Train Epoch: 5 [640/793 (80%)]	Loss: 0.264181

Test set: Average loss: 0.0088, Accuracy: 193/207 (93%)

Train Epoch: 6 [0/793 (0%)]	Loss: 0.384814
Train Epoch: 6 [640/793 (80%)]	Loss: 0.413308

Test set: Average loss: 0.0087, Accuracy: 193/207 (93%)

Train Epoch: 7 [0/793 (0%)]	Loss: 0.422129
Train Epoch: 7 [640/793 (80%)]	Loss: 0.246172

Test set: Average loss: 0.00

In [19]:
for epoch in range(1, 5+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['test'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.33

Train Epoch: 1 [0/793 (0%)]	Loss: 0.446304
Train Epoch: 1 [640/793 (80%)]	Loss: 0.401269

Test set: Average loss: 0.0097, Accuracy: 190/207 (92%)

Train Epoch: 2 [0/793 (0%)]	Loss: 0.509090
Train Epoch: 2 [640/793 (80%)]	Loss: 0.292777

Test set: Average loss: 0.0100, Accuracy: 193/207 (93%)

Train Epoch: 3 [0/793 (0%)]	Loss: 0.190211
Train Epoch: 3 [640/793 (80%)]	Loss: 0.533879

Test set: Average loss: 0.0089, Accuracy: 191/207 (92%)

Train Epoch: 4 [0/793 (0%)]	Loss: 0.569277
Train Epoch: 4 [640/793 (80%)]	Loss: 0.236288

Test set: Average loss: 0.0092, Accuracy: 192/207 (93%)

Train Epoch: 5 [0/793 (0%)]	Loss: 0.261837
Train Epoch: 5 [640/793 (80%)]	Loss: 0.508072

Test set: Average loss: 0.0086, Accuracy: 193/207 (93%)

